## MySQL code

In [141]:
from src.mysql import get_all_table_names, get_table_definition, get_knowledge_base_schema_for_table, query_random_rows

db_connection_params = {
    'host': 'localhost',
    'port': 3306,
    'user': 'daver',
    'password': 'pizzatime',
    'database': 'daver_db'
}

## Ollama Code

In [142]:
from ollama import ChatResponse
from ollama import Client


client = Client(
    # host='http://localhost:11434',
    host='http://192.168.178.38:11434',
    headers={'Content-Type': 'application/json'},
)
model = "deepseek-r1:8b"
options = {
    # "temperature": 0.2
    }

In [143]:
schema = {
  "type": "object",
  "properties": {
    "name":    {"type": "string"},
    "context": {"type": "string"},
    "fields": {
      "type": "array",
      "items": {
        "type":  "object",
        "properties": {
          "name":     {"type": "string"},
          "type":     {"type": "string"},
          "nullable": {"type": "string"},
          "key":      {"type": "string"},
          "default":  {"type": ["string", "null"]},
          "extra":    {"type": "string"},
          "context":  {"type": "string"}
        },
        "required": ["name", "context",]
      }
    }
  },
  "required": ["name", "context", "fields"]
}

In [144]:
system_message = {
            "role": "system",
            "content": (
                "You are a data analyst.\n"
                "Your ONLY task is to write a concise, human-readable knowledge base"
                "about the table represented by the columns and rows that you are shown.\n"
                "You will be given a sample of rows from the table to extract insights from. Don't make any comment about the number of rows you were given.\n"
                "When making numerical statements, consider this to be a sample of the entire table, therefore any statements you make can only be relative to the sample.\n"
                "Your output should be readable as knowledge base by another LLM."
                "• Never output code, markdown fences, or JSON.\n"
                "• Never describe the JSON format itself — only the database facts.\n"
                "• Some columns might be in Binary Format, or other formats that are not human readable. If you can distinguish the format, describe it, otherwise ignore it.\n"
                "If you are tempted to write code, STOP.\n"
                "Return ONLY a JSON object that matches the given schema. Fill in you knowledge base about the table in the top level 'context' field.\n"
                "Fill in the 'context' field in each of the 'fields' objects with your knowledge base about the column.\n"
                "Do not leave the 'context' field empty!"
            )
        },

In [145]:
example_A_user = {
  "name": "book_inventory",
  "context": "",
  "fields": [
    {"name": "isbn",       "type": "char(13)",     "nullable": "NO",  "key": "PRI", "default": None, "extra": "",            "context": ""},
    {"name": "title",      "type": "varchar(255)", "nullable": "NO",  "key": "",    "default": None, "extra": "",            "context": ""},
    {"name": "author",     "type": "varchar(255)", "nullable": "YES", "key": "",    "default": None, "extra": "",            "context": ""},
    {"name": "pub_year",   "type": "year",         "nullable": "YES", "key": "",    "default": None, "extra": "",            "context": ""},
    {"name": "in_stock",   "type": "int",          "nullable": "NO",  "key": "",    "default": 0,    "extra": "",            "context": ""}
  ]
}
example_A_assistant = {
  "name": "book_inventory",
  "context": "One row per title kept in the warehouse. Primary key is the 13-digit ISBN; titles are unique.",
  "fields": [
    {"name": "isbn",       "type": "char(13)",     "nullable": "NO",  "key": "PRI", "default": None, "extra": "", "context": "Global book identifier used as the primary key."},
    {"name": "title",      "type": "varchar(255)", "nullable": "NO",  "key": "",    "default": None, "extra": "", "context": "Human-readable title; duplicates possible across editions."},
    {"name": "author",     "type": "varchar(255)", "nullable": "YES", "key": "",    "default": None, "extra": "", "context": "Main author(s); blank for works in the public domain."},
    {"name": "pub_year",   "type": "year",         "nullable": "YES", "key": "",    "default": None, "extra": "", "context": "Four-digit publication year; helps with edition matching."},
    {"name": "in_stock",   "type": "int",          "nullable": "NO",  "key": "",    "default": 0,    "extra": "", "context": "Physical copies currently available for sale."}
  ]
}

example_B_user = {
  "name": "iot_sensor_data",
  "context": "",
  "fields": [
    {"name": "sensor_id",   "type": "varchar(40)", "nullable": "NO",  "key": "PRI", "default": None, "extra": "", "context": ""},
    {"name": "ts_utc",      "type": "timestamp",   "nullable": "NO",  "key": "PRI", "default": None, "extra": "", "context": ""},
    {"name": "temperature", "type": "float",       "nullable": "YES", "key": "",    "default": None, "extra": "", "context": ""},
    {"name": "humidity",    "type": "float",       "nullable": "YES", "key": "",    "default": None, "extra": "", "context": ""},
    {"name": "battery_mv",  "type": "smallint",    "nullable": "YES", "key": "",    "default": 0, "extra": "", "context": ""}
  ]
}
example_B_assistant = {
  "name": "iot_sensor_data",
  "context": "Time-series readings streamed from edge devices; composite primary key (sensor_id, ts_utc).",
  "fields": [
    {"name": "sensor_id",   "type": "varchar(40)", "nullable": "NO",  "key": "PRI", "default": None, "extra": "", "context": "UUID assigned to each deployed sensor."},
    {"name": "ts_utc",      "type": "timestamp",   "nullable": "NO",  "key": "PRI", "default": None, "extra": "", "context": "ISO timestamp in UTC when the packet was recorded."},
    {"name": "temperature", "type": "float",       "nullable": "YES", "key": "",    "default": None, "extra": "", "context": "Degrees Celsius; may be NULL if probe is faulty."},
    {"name": "humidity",    "type": "float",       "nullable": "YES", "key": "",    "default": None, "extra": "", "context": "Relative humidity percentage."},
    {"name": "battery_mv",  "type": "smallint",    "nullable": "YES", "key": "",    "default": None, "extra": "", "context": "Remaining battery in millivolts; values <3100 signal low-power state."}
  ]
}

example_C_user = {
  "name": "employee_time_off",
  "context": "",
  "fields": [
    {"name": "request_id",   "type": "bigint",       "nullable": "NO",  "key": "PRI", "default": None,    "extra": "auto_increment", "context": ""},
    {"name": "emp_id",       "type": "int",          "nullable": "NO",  "key": "MUL", "default": None,    "extra": "",              "context": ""},
    {"name": "start_date",   "type": "date",         "nullable": "NO",  "key": "",    "default": None,    "extra": "",              "context": ""},
    {"name": "end_date",     "type": "date",         "nullable": "NO",  "key": "",    "default": None,    "extra": "",              "context": ""},
    {"name": "reason_code",  "type": "char(3)",      "nullable": "YES", "key": "",    "default": "OTH",   "extra": "",              "context": ""},
    {"name": "approved_by",  "type": "int",          "nullable": "YES", "key": "",    "default": None,    "extra": "",              "context": ""},
    {"name": "approved_ts",  "type": "timestamp",    "nullable": "YES", "key": "",    "default": None,    "extra": "",              "context": ""}
  ]
}

example_C_assistant = {
  "name": "employee_time_off",
  "context": "Tracks vacation / sick-leave windows. One row per request; overlaps allowed until HR approval.",
  "fields": [
    {"name": "request_id",  "type": "bigint",    "nullable": "NO",  "key": "PRI", "default": None,  "extra": "auto_increment", "context": "Surrogate key issued by the application."},
    {"name": "emp_id",      "type": "int",       "nullable": "NO",  "key": "MUL", "default": None,  "extra": "",              "context": "Foreign key to employees table."},
    {"name": "start_date",  "type": "date",      "nullable": "NO",  "key": "",    "default": None,  "extra": "",              "context": "First calendar day away from work, inclusive."},
    {"name": "end_date",    "type": "date",      "nullable": "NO",  "key": "",    "default": None,  "extra": "",              "context": "Last calendar day away; must be ≥ start_date."},
    {"name": "reason_code", "type": "char(3)",   "nullable": "YES", "key": "",    "default": "OTH", "extra": "",              "context": "Enum: VAC, SCK, MAT, OTH … default means “other”."},
    {"name": "approved_by", "type": "int",       "nullable": "YES", "key": "",    "default": None,  "extra": "",              "context": "Manager’s emp_id once the request is approved."},
    {"name": "approved_ts", "type": "timestamp", "nullable": "YES", "key": "",    "default": None,  "extra": "",              "context": "UTC timestamp when approval was logged."}
  ]
}

In [146]:
import json

few_shot_examples = [
    # Example A
    {"role": "user",      "content": json.dumps(example_A_user)},
    {"role": "assistant", "content": json.dumps(example_A_assistant)},

    # Example B
    {"role": "user",      "content": json.dumps(example_B_user)},
    {"role": "assistant", "content": json.dumps(example_B_assistant)},

    # Example C
    {"role": "user",      "content": json.dumps(example_C_user)},
    {"role": "assistant", "content": json.dumps(example_C_assistant)},
]

In [147]:
person_schema = get_knowledge_base_schema_for_table('person', db_connection_params)

In [148]:
person_rows_df = query_random_rows('person', 100, db_connection_params)

Retrieved 100 rows from person


In [ ]:
# Get column names
columns = list(person_rows_df.columns)
max_rows = min(100, len(person_rows_df))  # Limit to 100 rows to avoid context overflow
rows = [person_rows_df.iloc[i].tolist() for i in range(max_rows)]

messages = [
        system_message,
        *few_shot_examples,
        {"role": "user", "content": "This is the table schema definition:\n" + json.dumps(person_schema)},
        *[{"role": "user", "content": f"This is row {i}: {row}"} for i, row in enumerate(rows)]
    ]

while True:
    response = client.chat(
        model=model,
        messages=messages,
        stream=False,
        options=options,
        format=schema
    )
        
    response_json = json.loads(response.message.content)
    context_missing = False
    if response_json['context'] == '':
        context_missing = True
    for field in response_json['fields']:
        if field['context'] == '':
            context_missing = True
            break
    if not context_missing:
        break
    print("Response is missing context")
    print(response_json)

print(response_json)


{'name': 'CREATE TABLE IF NOT EXISTS employee (id INT, first_name VARCHAR(50), last_name VARCHAR(50), age INT, height_cm INT)', 'context': "<user> has provided a list of rows with six columns each. The structure should be inferred as follows:\n\n- Column 1 appears to be an identifier or key.\n- Column 2 is the full name of a person.\n- Columns 3,4 and 5 are names and ages?\nNo, wait let me look again: \n\nEach row has six elements. Let's assume columns based on common database schema:\n\nGiven that there are two tables involved (employee and employee_details), the first table is likely 'employees' with id, name, age, height, and possibly other fields.\nHowever, looking at the data, it seems like each row represents an employee record. The second column has full names. This looks like a list of employees from fictional databases or examples.", 'fields': [{'name': 'ID', 'context': 'This is likely to be a unique identifier for the employee.', 'type': 'INT'}, {'name': 'Name', 'context': 'F